<a href="https://colab.research.google.com/github/neuroidss/Open-WebUI-Colab/blob/main/similar_articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install opendatasets
!pip install pandas

In [6]:
import opendatasets as od
import pandas

od.download("https://www.kaggle.com/datasets/Cornell-University/arxiv")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: neuroidss
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv


100%|██████████| 1.39G/1.39G [00:20<00:00, 72.1MB/s]


In [7]:
import os
import json
import pandas as pd

#category_map = {
#'cs.AI': 'Artificial Intelligence',
#'cs.CL': 'Computation and Language',
#'cs.CV': 'Computer Vision and Pattern Recognition',
#'cs.IR': 'Information Retrieval',
#'cs.LG': 'Machine Learning',
#}

#def clean_categories(x):
#    cat_text = ''
#    cat_list = x.split(' ')
#    for i, item in enumerate(cat_list):
#        cat_name = category_map[item]
#        if cat_name != 'Not available':
#            if i == 0: cat_text = cat_name
#            else:      cat_text = cat_text + ', ' + cat_name
#    cat_text = cat_text.strip()
#    return cat_text

def clean_text(x):
    new_text = x.replace("\n", " ")
    new_text = new_text.strip()
    return new_text

def clean_dataframe():
    columns = ['id', 'authors', 'title', 'categories', 'abstract']

    # Specify the folders containing the JSON and schema files
#    filepath = '/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json'
    filepath = 'arxiv/'+'arxiv-metadata-oai-snapshot.json'
    data = []

    # Open the file and read line by line
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            doc = json.loads(line)
            lst = [doc['id'], doc['authors'], doc['title'], doc['categories'], doc['abstract']]
            data.append(lst)

    arxiv_data = pd.DataFrame(data=data, columns=columns)
#    topics = ['cs.AI', 'cs.CV', 'cs.IR', 'cs.LG', 'cs.CL']

#    filtered_data                  = arxiv_data[arxiv_data['categories'].isin(topics)]
    filtered_data                  = arxiv_data
#    filtered_data['categories']    = filtered_data['categories'].apply(clean_categories)
    filtered_data['title']         = filtered_data['title'].apply(clean_text)
    filtered_data['abstract']      = filtered_data['abstract'].apply(clean_text)
    filtered_data['prepared_text'] = filtered_data['title'] + ' \n ' + filtered_data['abstract']
    return filtered_data

df = clean_dataframe()
df.to_csv('arxiv/'+'arxiv-metadata-oai-snapshot.csv', encoding='utf-8', index=False)


In [1]:
import pandas as pd

arxiv_data = pd.read_csv('arxiv/'+"arxiv-metadata-oai-snapshot.csv")

#topics = ['cs.AI', 'cs.CV', 'cs.IR', 'cs.LG', 'cs.CL']

#filtered_data                  = arxiv_data[arxiv_data['categories'].isin(topics)]
#filtered_data                  = arxiv_data
#filtered_data['prepared_text'] = filtered_data['title'] + ' \n ' + filtered_data['abstract']

#print(filtered_data.shape)

import pandas as pd

# Create a sample DataFrame
#df = pd.DataFrame({
#    'Name': ['Alice', 'Bob', 'Charlie'],
#    'Age': [25, 30, 35],
#    'Country': ['USA', 'Canada', 'Mexico']
#})

# Define a search function
def search_string(s, search):
    return search in str(s).lower()

filtered_arxiv_data = arxiv_data.loc[:, arxiv_data.columns == 'prepared_text']

# Search for the string 'al' in all columns
mask = filtered_arxiv_data.apply(lambda x: x.map(lambda s: search_string(s, 'eeg')))

# Filter the DataFrame based on the mask
filtered_arxiv_data = filtered_arxiv_data.loc[mask.any(axis=1)]

# Search for the string 'al' in all columns
mask = filtered_arxiv_data.apply(lambda x: x.map(lambda s: search_string(s, 'working')))

# Filter the DataFrame based on the mask
filtered_arxiv_data = filtered_arxiv_data.loc[mask.any(axis=1)]

# Search for the string 'al' in all columns
mask = filtered_arxiv_data.apply(lambda x: x.map(lambda s: search_string(s, 'memory')))

# Filter the DataFrame based on the mask
filtered_arxiv_data = filtered_arxiv_data.loc[mask.any(axis=1)]

print(filtered_arxiv_data)

filtered_arxiv_data.to_csv('arxiv/'+'arxiv-metadata-oai-snapshot_eeg_working_memory.csv', encoding='utf-8', index=False)

import os

filename = 'arxiv/'+"arxiv-metadata-oai-snapshot_working_memory/out.csv"
os.makedirs(os.path.dirname(filename), exist_ok=True)

for index, row in filtered_arxiv_data.iterrows():
    row.to_csv('arxiv/'+'arxiv-metadata-oai-snapshot_working_memory/out_{index}.csv'.format(index=index), encoding='utf-8', index=False)


<ipython-input-1-eb587e88e4bd>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  arxiv_data = pd.read_csv('arxiv/'+"arxiv-metadata-oai-snapshot.csv")


                                             prepared_text
671220   Filter Bank Common Spatial Patterns in Mental ...
721571   Comparison of Network Analysis Approaches on E...
756442   Assessing Functional Neural Connectivity as an...
858570   Decline of long-range temporal correlations in...
858571   The interplay between long- and short-range te...
952632   Use of Eye-Tracking Technology to Investigate ...
1189540  Decoding Working Memory Load from EEG with LST...
1256255  Inhibition and Set-Shifting Tasks in Central E...
1290981  EEG source localization analysis in epileptic ...
1311364  Application of statistical analysis to working...
1332709  Single-channel EEG features during n-back task...
1369789  BCI learning induces core-periphery reorganiza...
1427483  Coherence of Working Memory Study Between Deep...
1432301  Ridge-penalized adaptive Mantel test and its a...
1433999  A Pilot Study on Visually Stimulated Cognitive...
1484533  Cross-Subject Domain Adaptation for Classifyin.

In [ ]:
openwebui_host=''
token="sk-7c61dffdebe14dd992b0ecf2b9c4196f"

In [ ]:
import requests

def upload_file(token, file_path):
    url = 'http://'+openwebui_host+':3000/api/v1/files/'
    headers = {
        'Authorization': f'Bearer {token}',
        'Accept': 'application/json'
    }
    files = {'file': open(file_path, 'rb')}
    response = requests.post(url, headers=headers, files=files)
    return response.json()

In [ ]:
import requests

def add_file_to_knowledge(token, knowledge_id, file_id):
    url = f'http://'+openwebui_host+':3000/api/v1/knowledge/{knowledge_id}/file/add'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    data = {'file_id': file_id}
    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [ ]:
import os
file_ids=[]
for file in os.listdir('arxiv/'+'arxiv-metadata-oai-snapshot_working_memory'):
  if file.endswith(".csv"):
    print(os.path.join('arxiv/'+'arxiv-metadata-oai-snapshot_working_memory', file))
    file_path='arxiv/'+'arxiv-metadata-oai-snapshot_working_memory/'+file
    response_json = upload_file(token, file_path)
    print(response_json)
#    knowledge_id
    file_id=response_json['id']
    file_ids.append(file_id)
#    add_file_to_knowledge(token, knowledge_id, file_id)
#    break

In [ ]:
import requests

def chat_with_file(token, model, query, file_id):
    url = 'http://'+openwebui_host+':3000/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    payload = {
        'model': model,
        'messages': [{'role': 'user', 'content': query}],
        'files': [{'type': 'file', 'id': file_id}]
    }
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

def chat_with_files(token, model, query, file_ids):
    url = 'http://'+openwebui_host+':3000/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    files = []
    for file_id in file_ids:
        files.append({'type': 'file', 'id': file_id})
    payload = {
        'model': model,
        'messages': [{'role': 'user', 'content': query}],
        'files': files
    }
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

In [ ]:
model="deepseekr1-coder-14b"
query='make psytrance visuals, controlled by simulated eeg stream, based on working memory increase science, put css and js inside single html file. all comments leave only inside html block. no text output outside html block'
for file_id in file_ids:
#  response_json = chat_with_file(token, model, query, file_id)
  file_ids_plus = file_ids.copy()
  file_ids_plus.append(file_id)
  response_json = chat_with_files(token, model, query, file_ids_plus)

  print(response_json)
  import os

  filename = 'arxiv/'+"arxiv-metadata-oai-snapshot_working_memory/html/out.html"
  os.makedirs(os.path.dirname(filename), exist_ok=True)
  filename = 'arxiv/'+'arxiv-metadata-oai-snapshot_working_memory/html/out_'+file_id+'.html'
  f = open(filename, "w")
  response_message_content = response_json['choices'][0]['message']['content']
  created_html = ''
  for response_message_content_split in response_message_content.split("```"):
    split_html = response_message_content_split.split("html\n")
    if len(split_html)>1 and len(split_html[0])==0 and len(created_html)==0:
      created_html = split_html[1]
  f.write(created_html)
  f.close()
#  import IPython
#  IPython.display.HTML(filename=filename)
#  from IPython.display import IFrame
#  IFrame(src=filename, width=900, height=600)

In [ ]:
import requests

def chat_with_collection(token, model, query, collection_id):
    url = 'http://'+openwebui_host+':3000/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    payload = {
        'model': model,
        'messages': [{'role': 'user', 'content': query}],
        'files': [{'type': 'collection', 'id': collection_id}]
    }
    response = requests.post(url, headers=headers, json=payload)
    return response.json()